# Comparison of data


by soniame@iadb.org

- **IDB COVID Impact Dashboard**

Database: `spd_sdv_waze_corona.prod_daily_daily_index`


- **Google COVID-19 Mobility Report**

CSV: [Google link](https://www.google.com/covid19/mobility/)

Baseline: median value for the corresponding day of week during a 5 week period from January 3rd, 2020 to February 6th, 2020. 

- **Waze driven kilometers** driven kilometers percent change from baseline. 

CSV: [Waze link](https://www.waze.com/covid19)

Baseline: average value for the corresponding day of week during a 2 week period from February 11th, 2020 to February 25th, 2020.

In [2]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/soniame/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

import plotnine as p9
from mizani.breaks import date_breaks
from mizani.formatters import date_format
p9.theme_set(p9.theme_linedraw()) # default theme

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

# Manipulation 
from siuba import *

In [3]:
from src import utils
conn = utils.connect_athena(path='../configs/athena.yaml')

In [4]:
iso_codes = pd.read_sql_query("""
select 
    distinct country_iso_code, country_name
from spd_sdv_waze_corona.prod_daily_daily_index
""", conn)

In [5]:
len(iso_codes.country_iso_code.to_list())

20

In [6]:
#iso_codes.sort_values('country_iso_code')

## Country level

### Load data (country level)

Loading data per country. 

#### Waze dashboard

In [7]:
df_dash = pd.read_sql_query("""
select 
    *,
    (date_parse(concat(cast(year as varchar), ' ', cast(month as varchar), ' ', cast(day as varchar)), '%Y %m %e')) as date
from spd_sdv_waze_corona.prod_daily_daily_index
where region_type = 'country';
""", conn)
# Bolivia

In [8]:
#tab = (df_dash 
# >> group_by('country_iso_code', 'country_name', 'region_slug')
# >> summarize(min_date = _.date.min(),
#             max_date = _.date.max(),
#             n_obs = _.date.count())
# >> ungroup()
#)
#tab.to_csv('~/private/projects/google_mob_country_list.csv')
#tab

In [9]:
print(len(iso_codes.country_iso_code.to_list()))
#df_dash.head()
print(len(df_dash.country_iso_code.unique())) # Bolivia is missing
df_dash.sort_values('country_iso_code').country_iso_code.unique()

20
19


array(['AR', 'BB', 'BR', 'CL', 'CO', 'CR', 'DO', 'EC', 'GT', 'HN', 'JM',
       'MX', 'NI', 'PA', 'PE', 'PY', 'SV', 'TT', 'UY'], dtype=object)

In [10]:
#df_dash.columns.tolist()
print(df_dash.shape)
print(len(df_dash.country_name.unique()))
df_dash.country_name.unique()

(8828, 20)
19


array(['El Salvador', 'Mexico', 'Honduras', 'Dominican Republic',
       'Uruguay', 'Colombia', 'Panama', 'Nicaragua', 'Barbados',
       'Guatemala', 'Costa Rica', 'Ecuador', 'Jamaica', 'Peru',
       'Argentina', 'Brazil', 'Paraguay', 'Trinidad and Tobago', 'Chile'],
      dtype=object)

#### Waze driven km

In [11]:
df_waze = pd.read_csv("/home/soniame/shared/spd-sdv-omitnik-waze/corona/mobility/Waze _ COVID-19 Impact Dashboard_Country-Level Data_Table.csv", 
                      error_bad_lines=False)
df_waze.columns = ['date', 'country', 'pc_driven_miles']
df_waze.head(3)

b'Skipping line 19249: expected 3 fields, saw 4\nSkipping line 21046: expected 3 fields, saw 4\nSkipping line 21487: expected 3 fields, saw 4\n'


,date,country,pc_driven_miles
0,"Jun 29, 2021",Malaysia,-0.49
1,"Jun 29, 2021",Argentina,-0.41
2,"Jun 29, 2021",Indonesia,-0.40


In [12]:
df_waze['date'] = df_waze.date.apply(lambda x : datetime.datetime.strptime(x, '%b %d, %Y'))
df_waze['exists'] = df_waze.country.isin([ 'Argentina','Brazil', 'Chile', 'Colombia', 'Costa Rica',  
                                          'Ecuador', 'Guatemala',  'Mexico',  'Peru', 'Uruguay']) 
df_waze = df_waze.siu_filter(_.exists).siu_mutate(pc_driven_miles=100*(_.pc_driven_miles + 1 ))      
print(df_waze.shape)
df_waze.head()

(4877, 4)


,date,country,pc_driven_miles,exists
1,2021-06-29,Argentina,59.0,True
5,2021-06-29,Uruguay,68.0,True
7,2021-06-29,Peru,72.0,True
9,2021-06-29,Mexico,76.0,True
13,2021-06-29,Costa Rica,86.0,True


Missing countries: Panama, Honduras, Trinidad and Tobago, Dominican Republic, Paraguay, Barbados, El Salvador, Jamaica, Nicaragua

In [13]:
#tab = (df_waze 
# >> group_by('country')
# >> summarize(min_date = _.date.min(),
#             max_date = _.date.max(),
#             n_obs = _.date.count())
# >> ungroup()
#)
##tab.to_csv('~/private/projects/google_mob_country_list.csv')
#tab

In [14]:
print('Countries: ' + str(len(df_waze.country.unique())) )
df_waze.country.unique()

Countries: 10


array(['Argentina', 'Uruguay', 'Peru', 'Mexico', 'Costa Rica', 'Ecuador',
       'Brazil', 'Guatemala', 'Colombia', 'Chile'], dtype=object)

Turn baseline from 2020-03-02 to 2020-03-08 per day of week

In [15]:
df_waze = df_waze \
              .siu_select('country', 'date', 
                          'pc_driven_miles')
df_waze['dow'] = df_waze.date.apply(lambda x : datetime.datetime.strftime(x, '%w'))  
df_waze.head(2)

,country,date,pc_driven_miles,dow
1,Argentina,2021-06-29,59.0,2
5,Uruguay,2021-06-29,68.0,2


In [16]:
df_waze_base = df_waze[(df_waze.date >= '2020-03-02') & (df_waze.date <= '2020-03-08')] \
    .drop('date', 1) \
    .rename(columns = {'pc_driven_miles':'pc_driven_miles_baseline'})
df_waze_base.head(2)

,country,pc_driven_miles_baseline,dow
21125,Chile,65.0,0
21128,Argentina,84.0,0


In [17]:
df_waze_rebase = df_waze.merge(df_waze_base)
df_waze_rebase['pc_driven_miles'] = 100*(df_waze_rebase['pc_driven_miles']- df_waze_rebase['pc_driven_miles_baseline'])/df_waze_rebase['pc_driven_miles_baseline']
df_waze_rebase.head()
df_waze = df_waze_rebase.drop(['pc_driven_miles_baseline'], 1)

In [18]:
gg1 = (p9.ggplot(df_waze_rebase, 
           p9.aes(x='dow', y ='pc_driven_miles_baseline')) 
 + p9.geom_point()
 + p9.facet_wrap('country', scales='free_y') 
 + p9.labs(title='Driven miles per country for base period')
 + p9.theme(figure_size = (11, 11),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
) 
gg2 = (p9.ggplot(df_waze, 
           p9.aes(x='date', y ='pc_driven_miles', color = 'dow')) 
 + p9.geom_point()
 + p9.facet_wrap('country', scales='free_y') 
 + p9.labs(title='Driven miles per country')
 + p9.theme(figure_size = (11, 11),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
) 

In [19]:
#gg2

In [20]:
#df_waze.head(2)

#### Google mobility

In [ ]:
df_goo = pd.read_csv("~/shared/spd-sdv-omitnik-waze/corona/mobility/Global_Mobility_Report.csv")  
#df_goo = df_goo[(df_goo.date != '-5') | (df_goo.date.isna() == False) | (df_goo.date == 'nan')]


In [29]:
df_goo = df_goo[(df_goo.date != '-5')  ]
df_goo.sort_values('date').tail()
#df_goo[(df_goo.date == 'nan')]

MemoryError: Unable to allocate 362. MiB for an array with shape (8, 5937908) and data type object

In [44]:
df_goo['inregion'] = df_goo.country_region_code.isin(iso_codes.country_iso_code.to_list())
df_goo['date'] = df_goo.date.apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
df_goo = df_goo \
    .siu_filter(_.inregion) \
    .siu_filter( _.sub_region_1.isnull(), _.metro_area.isnull())   
print(df_goo.shape)


TypeError: strptime() argument 1 must be str, not float

In [ ]:
tab = (df_goo 
 >> group_by('country_region_code', 'country_region')
 >> summarize(min_date = _.date.min(),
             max_date = _.date.max(),
             n_obs = _.date.count())
 >> ungroup()
)
#tab

In [ ]:
#df_goo[(df_goo.date == '2020-03-09') & (df_goo.country_region_code == 'AR')]

In [ ]:
print(len(df_goo.country_region_code.unique()))
df_goo.country_region_code.unique()

In [ ]:
#df_goo.dtypes
#df_goo.columns

Turn baseline from 2020-03-02 to 2020-03-08 per day of week

In [ ]:
df_goo = df_goo \
              .siu_select('country_region_code', 'date', 
                          'transit_stations_percent_change_from_baseline',
                          'workplaces_percent_change_from_baseline') \
            .siu_mutate(transit_stations_percent_change_from_baseline = (_.transit_stations_percent_change_from_baseline + 100 ),
                       workplaces_percent_change_from_baseline = (_.workplaces_percent_change_from_baseline + 100 ))      
df_goo['dow'] = df_goo.date.apply(lambda x : datetime.datetime.strftime(x, '%w'))  

In [ ]:
#df_goo[(df_goo.dow == '0') & (df_goo.country_region_code == 'AR') & (df_goo.date >= '2020-03-02')][0:50]

In [ ]:
df_goo_base = df_goo[(df_goo.date >= '2020-03-02') & (df_goo.date <= '2020-03-08')] \
    .drop('date', 1) \
    .rename(columns = {'transit_stations_percent_change_from_baseline':'transit_baseline',
                      'workplaces_percent_change_from_baseline':'workplace_baseline'})
df_goo_base.head(2)
df_goo_base[(df_goo_base.country_region_code == 'AR') ]

In [ ]:
# Rebase percentage change to dow from 2020-03-02 to 2020-03-08
df_goo_rebase = df_goo.merge(df_goo_base)
df_goo_rebase['transit_stations_percent_change_from_baseline_rb'] = 100*(df_goo_rebase['transit_stations_percent_change_from_baseline']-df_goo_rebase['transit_baseline'])/df_goo_rebase['transit_baseline']
df_goo_rebase['workplaces_percent_change_from_baseline_rb'] = 100*(df_goo_rebase['workplaces_percent_change_from_baseline']- df_goo_rebase['workplace_baseline'])/df_goo_rebase['workplace_baseline']
df_goo_rebase.head()
df_goo = df_goo_rebase.drop(['workplace_baseline', 'transit_baseline'], 1)


In [ ]:
print(df_goo_rebase.shape)
#df_goo_rebase[(df_goo_rebase.date >= '2020-03-09')  & (df_goo_rebase.date <= '2020-03-15') & (df_goo_rebase.country_region_code == 'AR')]

In [ ]:
gg1 = (p9.ggplot(df_goo_base, 
           p9.aes(x='dow', y ='workplace_baseline')) 
 + p9.geom_point()
 + p9.facet_wrap('country_region_code', scales='free_y') 
 + p9.labs(title='Workplace per country for base period')
 + p9.theme(figure_size = (11, 11),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
) 
gg2 = (p9.ggplot(df_goo, 
           p9.aes(x='date', y ='workplaces_percent_change_from_baseline_rb', color = 'dow')) 
 + p9.geom_point()
 + p9.facet_wrap('country_region_code', scales='free_y') 
 + p9.labs(title='Workplace per country')
 + p9.theme(figure_size = (11, 11),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
) 
#print(gg2)

In [ ]:
df_goo[(df_goo.date >= '2020-03-09') & (df_goo.date <= '2020-03-15') & (df_goo.country_region_code == 'AR')]

### Comparison

In [ ]:
df = (df_dash \
     .siu_select('region_slug', 'country_name', 'country_iso_code', 'date', 'tcp') 
 >> left_join(_, 
          df_waze \
              .siu_select('country', 'date', 'pc_driven_miles') \
              .siu_rename(country_name='country'), 
              on=['country_name', 'date']) 
 >> left_join(_, 
          df_goo \
              .siu_select('country_region_code', 'date', 
                          'transit_stations_percent_change_from_baseline_rb',
                          'workplaces_percent_change_from_baseline_rb') \
              .siu_rename(country_iso_code='country_region_code',
                          workplace='workplaces_percent_change_from_baseline_rb',
                          transit='transit_stations_percent_change_from_baseline_rb'), 
              on=['country_iso_code', 'date'])
# >> left_join(_, 
#         df_max \
#              .siu_select('date', 'region_slug', 'tcp_clean'),
#              on=['region_slug', 'date'])
 >> gather('measure', 'value', _["tcp", "pc_driven_miles", 'workplace', "transit", 'tcp']) 
)
df.dtypes
df.head(2)

In [ ]:
(p9.ggplot(df.siu_filter(_.measure != 'tcp',
                        #_.measure != 'transit',
                         #_.date >= '2020-03-09',
                         #_.date <= '2020-03-16'
                        ), 
           p9.aes(x='date', y ='value', color = 'measure')) 
 + p9.geom_line(alpha = .7)
 + p9.facet_wrap('country_name', scales='free_y') 
 + p9.labs(title='Percent change per country')
 + p9.theme(figure_size = (11, 11),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
) 

In [ ]:
#df.siu_filter(_.country_name=='Mexico', _.measure == 'tcp').head()

In [ ]:
df_country = df

## City level

### Load data (city level)

In [ ]:
cities_names = (pd.read_csv('../docs/city_names_dictionary.csv') 
    >> filter(_.country_iso_code__dash.isnull() == False)
)
cities_names.head(2)

#### Waze dashboard

In [ ]:
df_dash = pd.read_sql_query("""
select 
    *,
    (date_parse(concat(cast(year as varchar), ' ', cast(month as varchar), ' ', cast(day as varchar)), '%Y %m %e')) as date
from spd_sdv_waze_corona.prod_daily_daily_index
where region_type = 'city';
""", conn)

In [ ]:
df_dash.shape

In [ ]:
tab = (df_dash 
 >> group_by('country_iso_code', 'country_name', 'region_slug', 'region_name')
 >> summarize(min_date = _.date.min(),
             max_date = _.date.max(),
             n_obs = _.date.count())
 >> ungroup()
)
tab.to_csv('~/private/projects/dash_mob_city_list.csv')
tab

In [ ]:
print(len(df_dash.region_name.unique())) # Bolivia is missing
df_dash.sort_values('region_name').region_name.unique()
df_dash.sort_values('region_slug').region_slug.unique()

#### Waze driven km

In [ ]:
df_waze = pd.read_csv("/home/soniame/shared/spd-sdv-omitnik-waze/corona/mobility/Waze _ COVID-19 Impact Dashboard_City-Level Data_Table.csv")
df_waze.sort_values('Country').Country.unique()

In [ ]:
df_waze = pd.read_csv("/home/soniame/shared/spd-sdv-omitnik-waze/corona/mobility/Waze _ COVID-19 Impact Dashboard_City-Level Data_Table.csv")
df_waze.columns = ['date', 'city', 'country', 'pc_driven_miles']
df_waze['date'] = df_waze.date.apply(lambda x : datetime.datetime.strptime(x, '%b %d, %Y'))
df_waze['exists'] = df_waze.country.isin([ 'Argentina','Brazil', 'Chile', 'Colombia', 'Costa Rica',  
                                          'Ecuador', 'Guatemala',  'Mexico',  'Peru', 'Uruguay']) 
df_waze = df_waze.siu_filter(_.exists).siu_mutate(pc_driven_miles=100*(_.pc_driven_miles+1))      

In [ ]:
tab = (df_waze 
 >> group_by('country', 'city')
 >> summarize(min_date = _.date.min(),
             max_date = _.date.max(),
             n_obs = _.date.count())
 >> ungroup()
)
tab.to_csv('~/private/projects/waze_mob_city_list.csv')
tab.head()

In [ ]:
print(len(df_waze.city.unique())) # Bolivia is missing
df_waze.sort_values('city').city.unique()

In [ ]:
df_waze.head(3)

Turn baseline from 2020-03-02 to 2020-03-08 per day of week

In [ ]:
df_waze = df_waze \
              .siu_select('city', 'country', 'date', 
                          'pc_driven_miles')
df_waze['dow'] = df_waze.date.apply(lambda x : datetime.datetime.strftime(x, '%w'))  
df_waze.head(2)

In [ ]:
df_waze_base = df_waze[(df_waze.date >= '2020-03-02') & (df_waze.date <= '2020-03-08')] \
    .drop('date', 1) \
    .rename(columns = {'pc_driven_miles':'pc_driven_miles_baseline'})
df_waze_base.head(2)

In [ ]:
df_waze_rebase = df_waze.merge(df_waze_base)
df_waze_rebase['pc_driven_miles'] = 100*(df_waze_rebase['pc_driven_miles']- df_waze_rebase['pc_driven_miles_baseline'])/df_waze_rebase['pc_driven_miles_baseline']
print(df_waze.shape)
print(df_waze_rebase.shape)
df_waze_rebase.head()
df_waze = df_waze_rebase.drop(['pc_driven_miles_baseline'], 1)

In [ ]:
df_waze.head(2)

#### Google mobility

In [ ]:
df_goo = pd.read_csv("~/shared/spd-sdv-omitnik-waze/corona/mobility/Global_Mobility_Report.csv")  


In [ ]:
df_goo['inregion']=df_goo.country_region_code.isin(iso_codes.country_iso_code.to_list())
df_goo['date'] = df_goo.date.apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
df_goo = df_goo \
    .siu_filter(_.inregion) \
    .siu_filter( _.sub_region_1.isnull()==False)#, _.metro_area.isnull())
df_goo.sub_region_2=df_goo.sub_region_2.fillna('None')
df_goo.metro_area=df_goo.metro_area.fillna('None')
print(df_goo.shape)

In [ ]:
tab = (df_goo 
 >> group_by('country_region_code', 'country_region', 'sub_region_1', 'sub_region_2', 'metro_area')
 >> summarize(min_date = _.date.min(),
             max_date = _.date.max(),
             n_obs = _.date.count())
 >> ungroup()
)
tab.to_csv('~/private/projects/google_mob_city_list.csv')
tab.head(2)

In [ ]:
df_goo.head(2)

In [ ]:
tab.shape

In [ ]:
df_goo = df_goo \
              .siu_select('country_region_code', 'country_region',
                          'sub_region_1', 'sub_region_2', 
                          'date', 
                          'transit_stations_percent_change_from_baseline',
                          'workplaces_percent_change_from_baseline') \
            .siu_mutate(transit_stations_percent_change_from_baseline = (_.transit_stations_percent_change_from_baseline + 100 ),
                       workplaces_percent_change_from_baseline = (_.workplaces_percent_change_from_baseline + 100 ))      
df_goo['dow'] = df_goo.date.apply(lambda x : datetime.datetime.strftime(x, '%w'))  

In [ ]:
df_goo_base = df_goo[(df_goo.date >= '2020-03-09') & (df_goo.date <= '2020-03-15')] \
    .drop('date', 1) \
    .rename(columns = {'transit_stations_percent_change_from_baseline':'transit_baseline',
                      'workplaces_percent_change_from_baseline':'workplace_baseline'})
df_goo_base.head(2)
#df_goo_base[(df_goo_base.country_region_code == 'AR') ]

In [ ]:
df_goo_rebase = df_goo.merge(df_goo_base)
df_goo_rebase['transit_stations_percent_change_from_baseline_rb'] = 100*(df_goo_rebase['transit_stations_percent_change_from_baseline']-df_goo_rebase['transit_baseline'])/df_goo_rebase['transit_baseline']
df_goo_rebase['workplaces_percent_change_from_baseline_rb'] = 100*(df_goo_rebase['workplaces_percent_change_from_baseline']- df_goo_rebase['workplace_baseline'])/df_goo_rebase['workplace_baseline']
print(df_goo.shape)
print(df_goo_rebase.shape)
df_goo_rebase.head()
df_goo = df_goo_rebase.drop(['workplace_baseline', 'transit_baseline'], 1)


### Comparison

Comparing all sources per region slug.

In [ ]:
df_goo.head(1)

In [ ]:
cities_names.head()

In [ ]:
df=df_dash \
.siu_select('country_iso_code', 'country_name', 'region_slug', 'region_name', 'date', 'tcp') \
.add_suffix('__dash') \
.merge(cities_names, how='inner') \
.merge(df_waze.add_suffix('__waze'), 
       how='left', 
       left_on=['country__waze','city__waze', 'date__dash'],
       right_on = ['country__waze','city__waze', 'date__waze']) \
.merge(df_goo.add_suffix('__goo'), 
       how='left', 
       left_on=['country_region_code__goo','country_region__goo', 
                'sub_region_1__goo', 'sub_region_2__goo', 'date__dash'],
       right_on =['country_region_code__goo','country_region__goo', 
               'sub_region_1__goo', 'sub_region_2__goo', 'date__goo']) \
.siu_rename(transit__goo='transit_stations_percent_change_from_baseline_rb__goo',
           workplace__goo='workplaces_percent_change_from_baseline_rb__goo') \
.siu_select('country_iso_code__dash', 'country_name__dash', 'region_slug__dash', 
        'date__dash', 'tcp__dash', 'pc_driven_miles__waze',
           'transit__goo', 'workplace__goo') \
.siu_gather('measure', 'value', _["tcp__dash":"workplace__goo"]) 



In [ ]:
print(df.shape)

In [ ]:
print(df_dash.shape)
print(df_waze.shape)
print(df_goo.shape)
print(df.shape)
df.country_iso_code__dash.unique()

In [ ]:
(p9.ggplot(df.siu_filter(_.country_iso_code__dash == 'MX', _.value <100), 
           p9.aes(x='date__dash', y ='value', color = 'measure')) 
 + p9.geom_line()
 + p9.facet_wrap(['country_iso_code__dash', 'region_slug__dash'], scales='free_y', ncol= 3) 
 + p9.theme(figure_size = (10, 10),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
 + p9.labs(title='Observed measure per city')
) 

In [ ]:
df_region=df

#### Rescaled percentage change to common week 2020-03-23

Modified baseline: avergae percentage for the week period from March 23th, 2020 to March 29th, 2020. 

In [ ]:
df_mean = (df
 >> filter(_.date__dash >= '2020-03-02', _.date__dash <= '2020-03-08')
 >> group_by('region_slug__dash', 'measure')
 >> summarize(mean_val = _.value.mean())
 >> ungroup()
)
df_re = (df.merge(df_mean.reset_index(), on=['region_slug__dash', 'measure'], how='left')
 >> mutate(value_rescaled = -100*_.value/_.mean_val)
)
df_re.country_iso_code__dash.unique()

In [ ]:
(p9.ggplot(df_re.siu_filter(_.country_iso_code__dash == 'MX', 
                            #_.region_slug__dash =='mexicocity',
                            #_.value_rescaled < 300,
                            #_.measure!='workplace__goo', _.measure!='transit__goo',
                            # _.date__dash >= '2020-11-01', _.date__dash < '2020-12-01'
                           ), 
           p9.aes(x='date__dash', y ='value_rescaled', color = 'measure')) 
 + p9.geom_line()
 + p9.facet_wrap(['country_iso_code__dash', 'region_slug__dash'], scales='free_y', ncol= 3) 
 + p9.labs(title='Rescaled baseline')
 + p9.theme(figure_size = (10, 10),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
) 

#### Moving average smoothing

Rolling average with previous seven days.

In [ ]:
df_sao = df_re.siu_filter(_.country_iso_code__dash == 'MX', 
                            #_.region_slug__dash =='riodejaneiro',
                            #_.value_rescaled < 300,
                            #_.measure!='workplace__goo', _.measure!='transit__goo',
                            # _.date__dash >= '2020-11-01', _.date__dash < '2020-12-01'
                           )
#df_sao.head()

In [ ]:
df_ma = (df_sao
 >> arrange('region_slug__dash', 'measure', 'date__dash')
 >> group_by('region_slug__dash', 'measure')
 >> mutate(value_rescaled_ma = _.value_rescaled.rolling(7).mean())
 >> ungroup()
)
df_ma.head()

In [ ]:
(p9.ggplot(df_ma, 
           p9.aes(x='date__dash', y ='value_rescaled_ma', color = 'measure')) 
 + p9.geom_line()
 + p9.facet_wrap(['country_iso_code__dash', 'region_slug__dash'], scales='free_y', ncol= 3)
 + p9.labs(title='Ppercent change smooth')
 + p9.theme(figure_size = (10, 10),
            axis_text_x=p9.element_text(rotation=90, hjust=1), 
            legend_position='top')
) 

In [ ]:
#df.siu_filter(_.region_slug__dash=='arequipa', _.date__dash >= '2020-03-23', _.date__dash <= '2020-03-29').value
#df_mean.siu_filter(_.region_slug__dash=='arequipa')
#df_waze.siu_filter(_.city=='Arequipa', _.date >= '2020-03-23', _.date <= '2020-03-29')

## Country + City

#### create DF union

In [ ]:
df_union = pd.concat([df_country, 
                     df_region.rename(columns={'country_iso_code__dash':'country_iso_code',
                          'country_name__dash':'country_name',
                          'region_slug__dash':'region_slug',
                          'date__dash':'date'})])
print(df.shape)
df_union.head()

In [ ]:
df_union.measure[df_union.measure == 'pc_driven_miles'] = 'pc_driven_miles__waze'
df_union.measure[df_union.measure == 'tcp'] = 'tcp__dash'
df_union.measure[df_union.measure == 'transit'] = 'transit__goo'
df_union.measure[df_union.measure == 'workplace'] = 'workplace__goo'

In [ ]:
df_union.measure.unique()

In [ ]:
df_union.siu_spread("measure", "value").to_csv("/home/soniame/shared/spd-sdv-omitnik-waze/corona/mobility/region_city_union.csv")


In [ ]:
len(df_union.region_slug.unique())

#### Read data frame union

In [ ]:
#df_union.siu_spread("measure", "value").to_csv("/home/soniame/shared/spd-sdv-omitnik-waze/corona/mobility/region_city_union.csv")
df_union = pd.read_csv("/home/soniame/shared/spd-sdv-omitnik-waze/corona/mobility/region_city_union.csv").drop('Unnamed: 0', axis=1)
df_union.head()

In [ ]:
df_union.describe()

reading daily data

In [ ]:
df_mobility = df_union#.siu_spread("measure", "value")
df_mobility['date'] = df_mobility.date.astype(str)

#df_daily = pd.read_csv("/home/soniame/shared/spd-sdv-omitnik-waze/corona/cleaning/daily/daily_daily_index_max.csv")
#df_daily = df_max[['date', 'region_slug', 'tci_observed', 'tcp_observed', 'tci_clean', 'tcp_clean']] \
#    .rename(columns = {'tci_clean':'tci_cleaned', 
#                       'tcp_clean':'tcp_cleaned'}) 

df_daily = pd.read_csv("/home/soniame/private/daily_index_index.csv")
df_daily.head()

In [ ]:
print(df_daily.columns)
print(df_mobility.columns)

In [ ]:
df_mobility_clean = df_daily.merge(df_mobility, 
                                 how = 'left')
df_mobility_clean.head(2)

In [ ]:
print(df_union.shape)
print(df_mobility.shape)
print(df_daily.shape)
print(df_mobility_clean.shape)

In [ ]:
print(sum(df_mobility_clean.pc_driven_miles__waze.isna())/len(df_mobility_clean.tcp__dash))
print(sum(df_mobility_clean.transit__goo.isna())/len(df_mobility_clean.tcp__dash))
print(sum(df_mobility_clean.workplace__goo.isna())/len(df_mobility_clean.tcp__dash))
print(sum(df_mobility_clean.tcp__dash.isna())/len(df_mobility_clean.tcp__dash))
df_mobility_clean.head(2)

In [ ]:
df_mobility_clean.to_csv("/home/soniame/private/smooth/daily_index_index_movility.csv")

In [ ]:
df_mobility_clean[df_mobility_clean.workplace__goo.isna() == False].region_slug.unique()

In [ ]:
print(len(df_mobility_clean[df_mobility_clean.workplace__goo.isna() == False].region_slug.unique()))
df_mobility_clean[df_mobility_clean.workplace__goo.isna() == False].region_slug.unique()

In [ ]:
print(len(df_mobility_clean[df_mobility_clean.pc_driven_miles__waze.isna() == False].region_slug.unique()))
df_mobility_clean[df_mobility_clean.pc_driven_miles__waze.isna() == False].region_slug.unique()


In [ ]:
df_mobility_clean.head(2)

In [ ]:
(p9.ggplot(df_mobility_clean.siu_filter(_.region_slug =='country_mexico'), 
           p9.aes(x='date', y ='tcp_observed', group= 'region_slug')) 
 + p9.geom_line()
 + p9.geom_line(p9.aes(y = 'tcp_cleaned'), color = 'gray')
 + p9.geom_line(p9.aes(y = 'tcp_cleaned_ls_20X'), color = 'red')
 + p9.geom_line(p9.aes(y = 'tcp_cleaned_ls_10X'), color = 'orange')
 +  p9.geom_line(p9.aes(y = 'tcp_cleaned_ls_15X'), color = 'blue')
 + p9.geom_line(p9.aes(y = 'tcp_cleaned_ls_OFF'), color = 'green')
 #+ p9.geom_line(p9.aes(y = 'workplace__goo'), color = 'red')
 #+ p9.geom_line(p9.aes(y = 'pc_driven_miles__waze'), color = 'blue')
 + p9.facet_wrap(['region_slug'], scales='free_y', ncol= 3) 
 + p9.labs(title='Mobility')
 + p9.theme(figure_size = (10, 6),
            axis_text_x=p9.element_text(rotation=90, hjust=1, size = 3), 
            legend_position='top')
) 

#### Daily in long format

In [ ]:
df_daily_long = (df_mobility_clean 
 >> select('date', 'region_slug', 
           'tci_observed',  'tci_cleaned', 
           'tci_cleaned_ls_4X', 'tci_cleaned_ls_20X', 
           'tci_cleaned_ls_10X', 'tci_cleaned_ls_15X',
           'tci_cleaned_ls_OFF', 'tci_cleaned_ls_MIX',
           'tcp_observed', 'tcp_cleaned', 
           'tcp_cleaned_ls_4X', 'tcp_cleaned_ls_20X', 
           'tcp_cleaned_ls_OFF', 'tcp_cleaned_ls_MIX', 
           'tci_cleaned_ls_10X', 'tci_cleaned_ls_15X',
           'pc_driven_miles__waze',
           'workplace__goo')
 >> gather('measure', 'value', _["tci_observed":"workplace__goo"])
)

df_daily_long['type'] = 'TCP'
tci_exist = df_daily_long['measure'].isin(['tci_observed',  'tci_cleaned', 
           'tci_cleaned_ls_4X', 'tci_cleaned_ls_20X', 
           'tci_cleaned_ls_10X', 'tci_cleaned_ls_15X',
           'tci_cleaned_ls_OFF', 'tci_cleaned_ls_MIX'])
df_daily_long['type'][tci_exist] = 'TCI'

print(df_daily_long.type.unique())
print(df_daily_long.measure.unique())
head(df_daily_long)

In [ ]:
df_daily_long.to_csv("/home/soniame/private/daily_daily_index_long.csv")